In [1]:
from tmilib import *

In [2]:

def get_users_with_tab_focus_data():
  users_with_tab_focus_data = []

  for user in list_users():
    if not sdir_exists('tab_focus_times_for_user/' + user + '.json'):
      continue
    tab_focus_times = get_tab_focus_times_for_user(user)
    ordered_visits = get_history_ordered_visits_for_user(user)
    start_time = tab_focus_times[0]['start']
    ordered_visits_after_start = [x for x in ordered_visits if x['visitTime'] > start_time]
    #print len(tab_focus_times)
    if len(tab_focus_times) > 2000 and len(ordered_visits_after_start) > 400:
      users_with_tab_focus_data.append(user)
  return users_with_tab_focus_data

#print len(users_with_tab_focus_data)

In [3]:
#user = 'dwCCD1uofp' # doing very poorly with this user
user = 'UrZvGUsI8N'
#user = users_with_tab_focus_data[0]
print user
#url_to_visits = get_history_visits_for_user(user)

UrZvGUsI8N


In [4]:
from session_tracker import get_focused_tab

In [5]:
def compute_tab_focus_times_with_events_for_user(user):
  current_session_tracker = SessionTracker(store_events=True)
  for line in get_log_with_mlog_active_times_for_user(user):
    current_session_tracker.process_input(uncompress_data_subfields(line))
  current_session_tracker.end_input()
  return current_session_tracker.get_output()

In [6]:
tab_focus_times_with_events = compute_tab_focus_times_with_events_for_user(user)


In [7]:
for line in get_log_with_mlog_active_times_for_user(user):
  data = uncompress_data_subfields(line)
  url = get_focused_tab(data)
  if url != None:
    print url
    print data
    break

https://tmi.stanford.edu/mturk3.html?username=A2YVYX9UOBU0BR
{u'url': u'https://tmi.stanford.edu/mturk3.html?username=A2YVYX9UOBU0BR', u'evt': u'mlog', u'time': 1458171960840.0}


In [8]:
print len(tab_focus_times_with_events)
tab_focus_times_with_events[0]

2694


{'active': 1458172164793.0,
 'end': 1458172165207.0,
 'events': [{'dup': True,
   u'evt': u'mlog',
   u'time': 1458171960840.0,
   u'url': u'https://tmi.stanford.edu/mturk3.html?username=A2YVYX9UOBU0BR'},
  {u'evt': u'mlog',
   u'time': 1458171977813.0,
   u'url': u'https://tmi.stanford.edu/mturk3.html?username=A2YVYX9UOBU0BR'},
  {u'evt': u'mlog',
   u'time': 1458171978814.0,
   u'url': u'https://tmi.stanford.edu/mturk3.html?username=A2YVYX9UOBU0BR'},
  {u'_id': {u'$oid': u'56e9f05949a95df57ca09895'},
   u'evt': u'window_focus_changed',
   u'ip': u'67.237.177.87',
   u'mturkid': u'A2YVYX9UOBU0BR',
   u'time': 1458171979166.0,
   'url': u'https://tmi.stanford.edu/mturk3.html?username=A2YVYX9UOBU0BR',
   u'user': u'UrZvGUsI8N',
   u'ver': u'2',
   u'windowid': 278},
  {u'_id': {u'$oid': u'56e9f05949a95df57ca09899'},
   u'evt': u'browser_focus_changed',
   u'ip': u'67.237.177.87',
   u'isfocused': True,
   u'mturkid': u'A2YVYX9UOBU0BR',
   u'time': 1458171979319.0,
   'url': u'https://tm

In [9]:
tab_focus_times = get_tab_focus_times_for_user(user)
print len(tab_focus_times)
print tab_focus_times[0]
#print tab_focus_times

2694
{u'url': u'https://tmi.stanford.edu/mturk3.html?username=A2YVYX9UOBU0BR', u'start': 1458171960840.0, u'end': 1458172165207.0, u'active': 1458172164793.0}


In [10]:
ordered_visits = get_history_ordered_visits_for_user(user)
#first_site = url_to_visits.keys()[0]
#print url_to_visits[first_site]

In [11]:
ordered_visits_after_start = []
for visit in ordered_visits:
  time = visit['visitTime']
  #if 1458171960840.0 <= time <= 1458172165207.0:
  #  print visit
  if time >= 1458171960840.0:
    ordered_visits_after_start.append(visit)


In [12]:
print len(ordered_visits_after_start)

212


In [13]:
class ReconstructFocusTimes:
  def __init__(self):
    self.output = []
    self.curitem = {}
    self.idleperiod = 60 # seconds
  def get_output(self):
    self.end_input()
    return self.output
  def end_input(self):
    if 'url' in self.curitem:
      last_active = self.curitem['active']
      self.curitem['end'] = last_active + self.idleperiod*1000
      self.output.append(self.curitem)
      self.curitem = {}
  def end_session(self, curtime):
    if 'url' in self.curitem:
      last_active = self.curitem['active']
      # ensures that end < last_active+60 seconds
      self.curitem['end'] = min(curtime, last_active + self.idleperiod*1000)
      #self.curitem['end'] = min(curtime, last_active + 1800*1000)
      self.output.append(self.curitem)
      self.curitem = {}
  def start_session(self, url, curtime):
    if url == None:
      raise Exception('start_session should not be called with url==None')
    self.end_session(curtime)
    # start: first event in the session
    # active: last event which was active in the session
    # end: when we believe the session ended
    self.curitem = {'url': url, 'start': curtime, 'active': curtime}
  def continue_session(self, url, curtime):
    if url == None:
      raise Exception('continue_session should not be called with url==None')
    if 'url' not in self.curitem:
      self.start_session(url, curtime)
      return
    prevurl = self.curitem['url']
    if url == prevurl: # still on same site
      # has it been less than 60 seconds since last activity?
      prev_active_time = self.curitem['active']
      if curtime < prev_active_time + 60*1000:
        self.curitem['active'] = curtime
        return
    # have gone to different site
    self.end_session(curtime)
    self.start_session(url, curtime)
  def process_history_line(self, data):
    url = data['url']
    curtime = data['visitTime']
    self.continue_session(url, curtime)

def to_visit_lengths(ordered_visits):
  focus_time_reconstructor = ReconstructFocusTimes()
  for visit in ordered_visits:
    focus_time_reconstructor.process_history_line(visit)
  return focus_time_reconstructor.get_output()

reconstructed_visit_lengths = to_visit_lengths(ordered_visits)

In [14]:
print len(reconstructed_visit_lengths)

11816


In [15]:
def get_earliest_start_time(visit_lengths):
  if len(visit_lengths) < 1:
    raise Exception('get_earliest_time called with empty list')
  return visit_lengths[0]['start']

def get_last_end_time(visit_lengths):
  if len(visit_lengths) < 1:
    raise Exception('get_last_visit_time called with empty list')
  return visit_lengths[-1]['end']

print get_earliest_start_time(reconstructed_visit_lengths)
print get_earliest_start_time(tab_focus_times)

print get_last_end_time(reconstructed_visit_lengths)
print get_last_end_time(tab_focus_times)

1.45065037512e+12
1.45817196084e+12
1.45826605607e+12
1.4587918075e+12


In [16]:
def ignore_all_before_start_or_after_end(visit_lengths, start_time, end_time):
  output = []
  for x in visit_lengths:
    if x['end'] < start_time:
      continue
    if x['start'] > end_time:
      continue
    #if x['start'] < start_time: # - 1000: # 1 second before or after
    #  continue
    #if x['end'] > end_time: # + 1000:
    #  continue
    output.append(x)
  return output

ref_start_time = max(get_earliest_start_time(tab_focus_times), get_earliest_start_time(reconstructed_visit_lengths))
ref_end_time = min(get_last_end_time(tab_focus_times), get_last_end_time(reconstructed_visit_lengths))
#ref_start_time = get_earliest_start_time(tab_focus_times) #+ 1000*3600*10
#ref_end_time = get_last_end_time(tab_focus_times)
#ref_end_time = get_last_end_time(tab_focus_times) - 1000*3600*48
#ref_end_time = ref_start_time + 1000*3600*11

evaluated_reconstructed_visits = ignore_all_before_start_or_after_end(reconstructed_visit_lengths, ref_start_time, ref_end_time)
evaluated_tab_focus_times = ignore_all_before_start_or_after_end(tab_focus_times, ref_start_time, ref_end_time)

In [17]:
print ref_start_time

1.45817196084e+12


In [18]:
print evaluated_tab_focus_times[0]

{u'url': u'https://tmi.stanford.edu/mturk3.html?username=A2YVYX9UOBU0BR', u'start': 1458171960840.0, u'end': 1458172165207.0, u'active': 1458172164793.0}


In [19]:
for x in reconstructed_visit_lengths:
  url = x['url']
  if 'tmi.stanford.edu' in url:
    print x
    break

{'url': u'https://tmi.stanford.edu/mturk3.html?username=A2YVYX9UOBU0BR', 'start': 1458171897703.717, 'end': 1458172011233.28, 'active': 1458171951233.28}


In [20]:
print len(evaluated_reconstructed_visits)
print len(evaluated_tab_focus_times)

184
321


In [21]:
def 

In [22]:
#print evaluated_reconstructed_visits[0]
#print evaluated_tab_focus_times[2]
#print evaluated_reconstructed_visits[-1]
#print evaluated_tab_focus_times[-1]

In [23]:
class UrlAsIndex:
  def __init__(self, offset=0):
    self.index_to_url = []
    self.url_to_index = {}
    for idx in range(offset):
      self.index_to_url.append(None)
  def get_index_for_url(self, url):
    if url in self.url_to_index:
      return self.url_to_index[url]
    index = len(self.index_to_url)
    self.index_to_url.append(url)
    self.url_to_index[index] = url
    return index
  def get_url_for_index(self, index):
    return self.index_to_url[index]

class UrlAtTime:
  def __init__(self, visits, start_time=None, end_time=None):
    if start_time == None:
      start_time = get_earliest_start_time(visits)
    self.offset = start_time # milliseconds
    if end_time == None:
      end_time = get_last_end_time(visits)
    end_time_minus_offset = (end_time - start_time) / 1000.0
    #self.output = [None]*int(round(end_time_minus_offset) + 1)
    self.output = numpy.zeros(int(round(end_time_minus_offset) + 1), dtype=int)
    self.url_as_index = UrlAsIndex(1)
    for visit in visits:
      self.process_visit(visit)
  def process_visit(self, visit):
    url = visit['url']
    url_idx = self.url_as_index.get_index_for_url(url)
    start = visit['start']
    end = visit['end']
    start_idx = int(round((start - self.offset) / 1000.0))
    if start_idx < 0:
      start_idx = 0
      #return
    if start_idx >= len(self.output):
      return
    end_idx = int(round((end - self.offset) / 1000.0))
    if end_idx < 0:
      return
    if end_idx >= len(self.output):
      end_idx = len(self.output) - 1
    for idx in range(start_idx, end_idx+1):
      self.output[idx] = url_idx
  def get_url_for_index(self, index):
    return self.url_as_index.get_url_for_index(index)
  def get_output(self):
    return self.output

def second_to_active_url(visits, start_time, end_time):
  # offset: millseconds
  #start_time = get_earliest_start_time(visits)
  #end_time = get_last_end_time(visits)
  url_at_time = UrlAtTime(visits, start_time, end_time)
  return url_at_time

result_reference = second_to_active_url(evaluated_tab_focus_times, ref_start_time, ref_end_time)
result_reconstructed = second_to_active_url(evaluated_reconstructed_visits, ref_start_time, ref_end_time)

In [24]:
print len(result_reference.get_output())
print len(result_reconstructed.get_output())

94096
94096


In [25]:
stats = Counter()
for ref_answer,rec_answer in zip(result_reference.get_output(), result_reconstructed.get_output()):
  if ref_answer == 0 and rec_answer == 0:
    stats['both_inactive'] += 1
  elif ref_answer == 0 and rec_answer != 0:
    stats['ref_inactive_but_rec_active'] += 1
  elif ref_answer != 0 and rec_answer == 0:
    stats['ref_active_but_rec_inactive'] += 1
  elif ref_answer != 0 and rec_answer != 0:
    ref_url = result_reference.get_url_for_index(ref_answer)
    rec_url = result_reconstructed.get_url_for_index(rec_answer)
    ref_domain = url_to_domain(ref_url)
    rec_domain = url_to_domain(rec_url)
    if ref_domain == rec_domain:
      if ref_url == rec_url:
        stats['correct_url'] += 1
      else:
        stats['correct_domain'] += 1
    else:
      stats['incorrect_domain'] += 1

In [26]:
print stats

Counter({'both_inactive': 88339, 'ref_active_but_rec_inactive': 2346, 'correct_url': 1774, 'ref_inactive_but_rec_active': 1183, 'incorrect_domain': 398, 'correct_domain': 56})


In [27]:
for ref_answer,rec_answer in zip(result_reference.get_output(), result_reconstructed.get_output())[:1000]:
  ref_domain = 'None'
  rec_domain = 'None'
  if ref_answer != 0:
    ref_url = result_reference.get_url_for_index(ref_answer)
    ref_domain = url_to_domain(ref_url)
  if rec_answer != 0:
    rec_url = result_reconstructed.get_url_for_index(rec_answer)
    rec_domain = url_to_domain(rec_url)
  print ref_domain, rec_domain

tmi.stanford.edu tmi.stanford.edu
tmi.stanford.edu tmi.stanford.edu
tmi.stanford.edu tmi.stanford.edu
tmi.stanford.edu tmi.stanford.edu
tmi.stanford.edu tmi.stanford.edu
tmi.stanford.edu tmi.stanford.edu
tmi.stanford.edu tmi.stanford.edu
tmi.stanford.edu tmi.stanford.edu
tmi.stanford.edu tmi.stanford.edu
tmi.stanford.edu tmi.stanford.edu
tmi.stanford.edu tmi.stanford.edu
tmi.stanford.edu tmi.stanford.edu
tmi.stanford.edu tmi.stanford.edu
tmi.stanford.edu tmi.stanford.edu
tmi.stanford.edu tmi.stanford.edu
tmi.stanford.edu tmi.stanford.edu
tmi.stanford.edu tmi.stanford.edu
tmi.stanford.edu tmi.stanford.edu
tmi.stanford.edu tmi.stanford.edu
tmi.stanford.edu tmi.stanford.edu
tmi.stanford.edu tmi.stanford.edu
tmi.stanford.edu tmi.stanford.edu
tmi.stanford.edu tmi.stanford.edu
tmi.stanford.edu tmi.stanford.edu
tmi.stanford.edu tmi.stanford.edu
tmi.stanford.edu tmi.stanford.edu
tmi.stanford.edu tmi.stanford.edu
tmi.stanford.edu tmi.stanford.edu
tmi.stanford.edu tmi.stanford.edu
tmi.stanford.e

In [28]:
#for x in evaluated_reconstructed_visits:
#  print x

In [29]:
#for x in evaluated_tab_focus_times:
#  print x

In [30]:
for x in reconstructed_visit_lengths:
  url = x['url']
  if url == 'https://www.mturk.com/mturk/continue?hitId=3ZY8KE4ISJ42CAV8NVJ18F1PI9SQVJ':
    print x

In [31]:
for x in tab_focus_times:
  url = x['url']
  if url == 'https://www.mturk.com/mturk/continue?hitId=3ZY8KE4ISJ42CAV8NVJ18F1PI9SQVJ':
    print x

In [32]:

'''
def to_visit_lengths(ordered_visits):
  output = []
  for item,idx in ordered_visits:
    url = item['url']
    visit_time = item['visitTime'] # timestamp in milliseconds
    result = {'url': url, 'start': visit_time, 'active': visit_time}
    visit_duration = 30*1000 # 30 seconds in milliseconds
    if idx < len(ordered_visits):
      next_item = ordered_visits[idx+1]
      next_visit_time = next_item['visitTime']
      time_until_next_visit = next_visit_time - visit_time
      if visit_duration > time_until_next_visit: # more than 30 minutes until next visit
        
      visit_duration = min(visit_duration, time_until_next_visit)
    output.append({'url': url, 'start': visit_time, })
'''

"\ndef to_visit_lengths(ordered_visits):\n  output = []\n  for item,idx in ordered_visits:\n    url = item['url']\n    visit_time = item['visitTime'] # timestamp in milliseconds\n    result = {'url': url, 'start': visit_time, 'active': visit_time}\n    visit_duration = 30*1000 # 30 seconds in milliseconds\n    if idx < len(ordered_visits):\n      next_item = ordered_visits[idx+1]\n      next_visit_time = next_item['visitTime']\n      time_until_next_visit = next_visit_time - visit_time\n      if visit_duration > time_until_next_visit: # more than 30 minutes until next visit\n        \n      visit_duration = min(visit_duration, time_until_next_visit)\n    output.append({'url': url, 'start': visit_time, })\n"